Creates a .vec file with regular intervals for lab data with irregular time stamps. 

In [7]:
import csv
import datetime
import tkinter as tk

from dateutil.parser import parse
from tkinter import filedialog
from tkinter import messagebox

In [8]:
def read_csv_as_nested_list(filename, separator, quote):
    """
    Inputs:
      filename  - Name of CSV file
      keyfield  - Field to use as key for rows
      separator - Character that separates fields
      quote     - Character used to optionally quote fields

    Output:
      Returns a list of lists where the outer list corresponds to a each 
      row in the CSV file. The inner lists contain each comma separated 
      value from the CSV file.
    """
    
    list1 = []
    with open(filename, newline='') as csvfile:
        csvreader = csv.reader(
            csvfile, delimiter=separator, quotechar=quote)
        for row in csvreader:
            list1.append(row)
            
    return list1

In [9]:
def create_lab_dict(list1, interval_s):
    """
    Input: List of timestamps and data starting at 3rd row and desired 
        vector frequency
    Output: A tuple, first a dictionary from the input list of timestamps 
            and lab data, second a parameter dictionary
    """
    lab_dict = {}
    for idx in range(3,len(list1)):
        lab_dict[parse(list1[idx][0])] = [list1[idx][1]]
        
    params = {}
    params['tag'] = list1[0][1]
    params['desc'] = list1[1][1]
    params['units'] = list1[2][1]
    first_timestamp = list1[3][0]
    last_timestamp = list1[len(list1)-1][0]
    
    delta = parse(last_timestamp) - parse(first_timestamp)
    interval = datetime.timedelta(seconds = interval_s)
        
    params['first_timestamp'] = parse(first_timestamp)
    params['last_timestamp'] = parse(last_timestamp)
    params['delta'] = delta
    params['interval'] = interval
    params['samples'] = int(delta/interval)
    
    return lab_dict, params

In [16]:
def create_data_list(lab_dict, params):
    """
    Input: A tuple, first a dictionary of datetime timestamps and lab data
            and second a parameter dictionary
    Output: A list of items to be written to the .vec file
    """
    timestamps = {}
    for idx in range(0,params['samples']):
        timestamp = params['first_timestamp'] + params['interval'] * idx
        
        if timestamp in lab_dict:
            timestamps[timestamp] = lab_dict[timestamp][0]
            last_good = lab_dict[timestamp][0]
        else:
            timestamps[timestamp] = last_good
            
    header = []
    header.append(len(timestamps))
    header.append(params['tag'])
    
    spaces = 42 - len(params['desc'])
    ts = params['first_timestamp'].strftime("%d-%b-%Y  %H:%M:%S")
    header.append(params['desc']+' '*spaces+ts)
    header.append(params['units']+' '*47+str(params['interval'].seconds)+".00 (SEC)")
    
    output = []
    for item in range(0,len(header)):
        output.append([header[item]])
    
    for timestamp in timestamps:
        output.append([timestamps[timestamp]])
    
    return output

In [17]:
def write_vec_file(file_name):
    """
    Input: .csv file path containing lab data starting at 3rd element
    Action: writes a .vec file for lab data with specified intervals
    """
    vec_str = file_name.split(".")
    vec_file_name = "{}.vec".format(vec_str[0])
    
    list1 = read_csv_as_nested_list(file_name, "\t","'")
    lab_dict, params = create_lab_dict(list1, 60)
    output = create_data_list(list1, lab_dict, params)
    
    with open(vec_file_name, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
    
        for w in range(0,len(output)):
            writer.writerow(output[w])

In [12]:
def multi_file_conversion():
    """
    Action: 
        Prompts user to select CSV files for conversion
        Converts selected CSV files to VEC files
    """
    root = tk.Tk()
    root.withdraw()

    file_path = filedialog.askopenfilenames(title = "Select file", filetypes=[("Test files", ".txt"),("CSV files", ".csv")])

    messages = ""
    fail_msg = ""
    
    for files in file_path:
        try:
            write_vec_file(files)
            messages += "{} conversion complete\n".format(files)
        except:
            fail_msg += "{} conversion failed\n".format(files)
            continue
    
    messagebox.showinfo(title = "Conversion results", message = messages)
    
    if fail_msg != "":
        messagebox.showerror(title = "Conversion failed", message = fail_msg)

In [15]:
multi_file_conversion()